In [1]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/761vyj1b1497641946937
ls

In [2]:
%scala
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
val myManualSchema = new StructType(Array(
new StructField("DEST_COUNTRY_NAME", StringType, true),
new StructField("ORIGIN_COUNTRY_NAME", StringType, true),
new StructField("count", LongType, false)
))

spark.read.format("csv")
.option("header", "true")
.option("mode", "FAILFAST")
.schema(myManualSchema)
.load("/FileStore/tables/761vyj1b1497641946937/2010_summary-506d8.csv")
.take(5)

In [3]:
%scala
val csvFile = spark.read.format("csv")
.option("header", "true")
.option("mode", "FAILFAST")
.schema(myManualSchema)
.load("/FileStore/tables/761vyj1b1497641946937/2010_summary-506d8.csv")

In [4]:
%sh cd ..
cd ..
cd dbfs/tmp
ls

In [5]:
%scala
csvFile.write.format("csv")
.mode("overwrite")
.option("sep", "\t")
.save("/tmp/my-tsv-file.tsv")

In [6]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/nrptmpui1497642469139
ls

In [7]:
%scala
spark.read.format("json")
.option("mode", "FAILFAST")
.schema(myManualSchema)
.load("/FileStore/tables/nrptmpui1497642469139/2010_summary-506d8.json")
.show(5)

In [8]:
%scala
csvFile.write.format("json")
.mode("overwrite")
.save("/tmp/my-json-file.json")

In [9]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/6jc7prea1497642653485
ls

In [10]:
%scala
spark.read.format("parquet")
.load("/FileStore/tables/6jc7prea1497642653485/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz-11168.parquet")
.show(5)

In [11]:
%scala
csvFile.write.format("parquet")
.mode("overwrite")
.save("/tmp/my-parquet-file.parquet")

In [12]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/pqifqj771497642823964
ls

In [13]:
%scala
spark.read.format("orc")
.load("/FileStore/tables/pqifqj771497642823964/part_r_00000_2c4f7d96_e703_4de3_af1b_1441d172c80f_snappy-724f5.orc")
.show(5)

In [14]:
%scala
csvFile.write.format("orc")
.mode("overwrite")
.save("/tmp/my-json-file.orc")

In [15]:
%sh cd ..
cd ..
cd dbfs/FileStore/tables/2ucaskql1497643167427
ls

In [16]:
%scala
val props = new java.util.Properties
props.setProperty("driver", "org.sqlite.JDBC")
val path = "FileStore/tables/2ucaskql1497643167427/my_sqlite-e9c7d.db"
val url = s"jdbc:sqlite://FileStore/tables/2ucaskql1497643167427/my_sqlite-e9c7d.db"
val tablename = "flight_info"
props.setProperty("driver", "org.sqlite.JDBC") // set to postgres
// dbProperties.setProperty("username", "some-username")
// dbProperties.setProperty("password", "some-password")
val hostname = "192.168.1.5"
val port = "2345"
// we would set a username and a password in the properties file.
val dbDatabase = "DATABASE"
val dbTable = "test"

In [17]:
%scala
import java.sql.DriverManager
val connection = DriverManager.getConnection(url)
connection.isClosed()
connection.close()

In [18]:
%scala
val dbDataFrame = spark.read.jdbc(url, tablename, props)
dbDataFrame.select("DEST_COUNTRY_NAME").distinct().show(5)

In [19]:
%scala
dbDataFrame.select("DEST_COUNTRY_NAME").distinct().explain

In [20]:
%scala
dbDataFrame
.filter("DEST_COUNTRY_NAME in ('Anguilla', 'Sweden')")
.explain

In [21]:
%scala
val pushdownQuery = """
(SELECT DISTINCT(DEST_COUNTRY_NAME)
FROM flight_info) as flight_info
"""
val dbDataFrame = spark.read.jdbc(url, pushdownQuery, props)

In [22]:
%scala
val predicates = Array(
"DEST_COUNTRY_NAME = 'Sweden'",
"DEST_COUNTRY_NAME = 'United States'")
val dbDataFrame = spark.read.jdbc(url, tablename, predicates, props)

In [23]:
%scala
dbDataFrame.rdd.getNumPartitions
dbDataFrame.select("DEST_COUNTRY_NAME").distinct().show()

In [24]:
%scala
spark.read.jdbc(url, tablename, props).count()
val predicates = Array(
"DEST_COUNTRY_NAME != 'Sweden'",
"Origin_COUNTRY_NAME != 'United States'")
spark.read.jdbc(url, tablename, predicates, props).count()

In [25]:
%scala
val colName = "count"
val lowerBound = 0L
val upperBound = 348113L // this is the max count in our database
val numPartitions = 10

In [26]:
%scala
spark.read.jdbc(url,
tablename,
colName,
lowerBound,
upperBound,
numPartitions,
props)
.count()

In [27]:
%scala
val newPath = "jdbc:sqlite://tmp/my-sqlite.db"
csvFile.write.mode("overwrite").jdbc(newPath, tablename, props)

In [28]:
%scala
spark.read.jdbc(newPath, tablename, props).count()

In [29]:
%scala
csvFile.write.mode("append").jdbc(newPath, tablename, props)
spark.read.jdbc(newPath, tablename, props).count()

In [30]:
%scala
spark.read.textFile("/tmp/five-csv-files.csv")
.map(stringRow => stringRow.split(","))
.show()
spark.read.textFile("/tmp/five-csv-files.csv")
.write.text("/tmp/my-text-file.txt")

In [31]:
%scala
csvFile.write
.mode("overwrite")
.partitionBy("DEST_COUNTRY_NAME")
.save("/tmp/partitioned-files.parquet")

In [32]:
%scala
val numberBuckets = 10
val columnToBucketBy = "count"
csvFile.write.format("parquet")
.mode("overwrite")
.bucketBy(numberBuckets, columnToBucketBy)
.saveAsTable("bucketedFiles")